In [6]:
import pandas as pd 
from tqdm import tqdm 
from pathlib import Path
from typing import Dict, List
import re 
from collections import defaultdict
import os
import re
import logging

from dotenv import load_dotenv
import openai
from tqdm import tqdm
import pandas as pd

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
output_handler = logging.StreamHandler()
output_handler.setLevel(logging.INFO)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

load_dotenv('../openai.env')
openai.api_key  = os.getenv('API_KEY')
logger = logging.getLogger()

### Utils

In [85]:
def min_to_second(string: str):
    """
    Перевеод формата времени матча в секунды
    
    Аргументы:
        string: str - формат времени матча 
    Возвращает:
        время в секундах (int)
    """
    
    if isinstance(string, str):
        pattern = r"(\d+)\.(\d+):(\d+)"

        match = re.match(pattern, string)
        if match:
            minutes = int(match.group(1))*60
            seconds = int(match.group(3))
            return minutes+seconds
    else:
        return 0
    
    
def get_file_list(path_docs: str) -> List[str]:
    """
    Получитьл список всех файлов в папке
    
    Аргументы:
        path_docs - путь до папки с исходниками для рассчёта статистики  
    Возвращает:
        список путей исходников
    """
    
    path = Path(path_docs)
    parent =  path.parent
    name = path.name
    return [parent/name/path for path in os.listdir(path)]

class StatsHolder:
    """
    Класс для хранение и обработки сигнальных показателей
    """
    
    target_columns: list= ['MIN', 'FGM','FGA','FG3M','FTM','FTA','FT_PCT', 
                           'OREB','DREB','REB','AST','STL','BLK','TO','PF','PTS', 'PLAYER_ID', 'GAME_ID']
    calculus_columns: list = ['MIN', 'FGM','FGA','FG3M','FTM','FTA',
                              'FT_PCT','OREB','DREB','REB','AST','STL','BLK','TO','PF','PTS']
    labels_columns: list = ['PLAYER_ID', 'GAME_ID']
    
    def __init__(self, players_stats: Dict[int, List[pd.Series]]):
        """
        Аргументы:
            players_stats - статистика по игрокам. Ключи - индексы игроков, 
                            значения - сисок статистики по игроку за все указанные игры
        """
        
        self.players_stats: Dict[int, List[pd.Series]] = players_stats # статистика по игрокам
        self._old_rating = self._calculate_rating() # инициализация начального рейтинга
        self._new_rating = None

    
    @classmethod
    def from_csv(cls, path_docs: str) -> 'StatsHolder':
        """ 
        Создание объекта StatsHolder из списка файлов со статистикой формата .csv
        """
       
        file_list = get_file_list(path_docs)[:100]
        players_stats = defaultdict(list)
        
        for path in tqdm(file_list, total=len(file_list)):
            df = pd.read_csv(path).fillna(0)
            
            if 'PLAYER_ID' not in df.columns:
                continue
            
            df['MIN'] = df['MIN'].apply(min_to_second)
            df[cls.calculus_columns] =df[cls.calculus_columns].fillna(0).astype(int)
            for index, row in df.iterrows():
                players_stats[row.PLAYER_ID].append(row[cls.target_columns])
                
        return cls(players_stats=players_stats)
        
        
    def add_record(self, record: pd.DataFrame) -> None:
        """
        Добавить запись к общецй статистике игроков
        
        Аргументы:
            record: pd.DataFrame - запись конкретной игры
        """
        
        for _, row in record[self.target_columns].iterrows():
            player_id = row.PLAYER_ID
            game_id = row.GAME_ID

            id_game_list = [stat.GAME_ID for stat in  self.players_stats[player_id]]
             
            if game_id in id_game_list:
                logger.info(f'This GAME_ID:{game_id} is already in the dataset') 
                return None
            
            self.players_stats[player_id].append(row)
           
        logger.info(f'added game_id: {game_id}') 
        
        if self._new_rating: # если новый рейтинг уже существует
            # новый рейтинг становится старым
            self._old_rating = self._new_rating
        # вычисляется нового рейтинга (после добавление записи)
        self._new_rating = self._calculate_rating() 
        
              
    def set_strategy(self, type_strategy: str='top10'):
        """
        Установить стратегию срабатывания сигнальных показателей
        
        Аргументы:
            type_strategy - типы стратегии
                может принимать значения:
                    - top<N>- анализирует изменение рейтинга относительно первых N записей
        """
        result ={'in_top':{}, 
                'out_top':{}}
        
        # если проверяем ТОПы
        if 'top' in type_strategy:
            N = int(type_strategy.replace('top','')) # размер топа
            old_rating = {indicator: value.iloc[:N]  for indicator, value in self._old_rating.items()}
            new_rating = {indicator: value.iloc[:N] for indicator, value in self._new_rating.items()}
            
            old_top = {col: set(old_rating[col]['PLAYER_ID'].to_list()) for col in old_rating.keys()}
            new_top = {col: set(new_rating[col]['PLAYER_ID'].to_list()) for col in new_rating.keys()}
            
       
        for col in self.calculus_columns:
                in_top =  new_top[col] - old_top[col]
                out_top = old_top[col] -  new_top[col]
                
                if in_top:
                        result ['in_top'][col] = in_top
                if out_top:
                        result ['out_top'][col] = out_top

        return result 
    
    def _calculate_rating(self) -> Dict[str, pd.DataFrame]:
        """
        Рассчёт рейтинга для явсе игроков по всем показателям
        
        Возвращает:
            словарь, где:
                ключь - это наименование сигнального показателя (MIN, PTS и т.п.)
                значение - это DataFrame со следующими колонками:
                    PLAYER_ID - идентификатор игрока
                    VALUE - суммарное значение показателя игрока за весь период рассчёта
                    RATING - какое место занимает игрок в рейтинге по этому показателю \
                        относительно других игроков (0 - самое высокое место)            
        """
        
        # агрегация (суммирование) по игрокам 
        df_all = pd.concat([pd.DataFrame(records) for _, records in self.players_stats.items()], axis=0)
        df_all = df_all.groupby('PLAYER_ID').sum()
        #  получение словаря с рейтингами игроков для каждого сигнального показателя
        ratings = {col: df_all[col].sort_values(ascending=False).to_frame().reset_index()\
                    .rename(columns={col:'VALUE'}).assign(RATING=range(len(df_all[col]))) for col in df_all.columns} 
                                                            
        return ratings

In [86]:
# Инициализация объекта
stats = StatsHolder.from_csv(path_docs='resource/boxscoretraditionalv2/')

100%|██████████| 100/100 [00:02<00:00, 37.55it/s]


In [88]:
# Создание новой тестовой записи об игре (которая должна поменять рейтинг по индикатору MIN)
record = pd.read_csv(r'resource\boxscoretraditionalv2\boxscoretraditionalv2_0_0012100003.csv') 
record['MIN'] = record['MIN'].apply(min_to_second)
record[stats.calculus_columns] = record[stats.calculus_columns].fillna(0).astype(int)
record['GAME_ID'] = 21211221
record.loc[0,'PLAYER_ID'] = 202681
record.loc[0,'MIN'] = 10000000000

# добавление записи
stats.add_record(record)


C:\Users\b.bulatov\AppData\Local\Temp\ipykernel_15416\2005501247.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '10000000000' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  record.loc[0,'MIN'] = 10000000000
2023-10-24 17:32:32,609 - INFO - This GAME_ID:21211221 is already in the dataset


In [93]:
# получаем данные по изменению топов

stats.set_strategy('top5')

{'in_top': {'MIN': {202681},
  'AST': {1627832},
  'BLK': {1629626},
  'TO': {1628369}},
 'out_top': {'MIN': {1628366},
  'AST': {1627749},
  'BLK': {201572},
  'TO': {1629012}}}